Попробовала дообучить **rugpt3small_based_on_gpt2** на датасете **recipes 27к** c kaggle https://www.kaggle.com/datasets/coolonce/recipes-and-interpretation-dim

Модель была обучена командой SberDevices с длиной последовательности 1024 с использованием преобразователей на 80B токенах примерно в 3 эпохи.
Класс задач: Text Generation

Обучала на QA c использованием 100% данных колонок name(название рецепта) и Инструкции(описание рецепта), дообучила на 25 эпох.
Этот вариант модлеи будет заготовкой одного из интентов (повар) для курсового проекта.

In [1]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 3.0 MB/s eta 0:00:00


In [2]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.8 MB/s eta 0:00:00


In [3]:
!pip install datasets transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.33.1
    Uninstalling transformers-4.33.1:
      Successfully uninstalled transformers-4.33.1


In [4]:
import numpy as np
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import torch
from sklearn.model_selection import train_test_split



In [5]:
import pandas as pd

In [6]:
df_rec = pd.read_csv('all_recepies_inter.csv', sep='\t')

In [40]:
df_rec

,Unnamed: 0,name,composition,cooking_type,Инструкции,dish_type,Дата,photo,source,composition_inter
0,0,рассольник классический с перловкой и солеными...,"[{'Перловка': 0.1, 'unit': 'стак. (200 мл)'}, ...","варка,жарка",Подготовить указанные ингредиенты для приготов...,первое,05.06.2015,photo_1000menu_1.jpg,https://1000.menu/cooking/33395-rassolnik-s-pe...,"[{'product_id': 4253, 'name_source': 'Перловая..."
1,1,Суп пюре из белокочаной капусты,"[{'Капуста белокочанная': 50.0, 'unit': 'гр'},...",варка,"Необходимые ингредиенты\r\nНарезаем лук, морко...",первое,27.06.2015,photo_1000menu_2.jpg,https://1000.menu/cooking/25399-sup-pure-iz-be...,"[{'product_id': 2286, 'name_source': 'Капуста ..."
2,2,Постные щи из квашеной капусты,"[{'Капуста квашеная': 116.7, 'unit': 'гр'}, {'...","варка,жарка,тушение","Честно признаюсь, у меня не было репы на момен...",первое,12.02.2013,photo_1000menu_3.jpg,https://1000.menu/cooking/5159-postnje-shchi,"[{'product_id': 0, 'name_source': 'Капуста ква..."
3,3,Тюря- простой суп быстро и вкусно,"[{'Квас': 0.2, 'unit': 'л'}, {'Лук репчатый': ...",сырое,"\r\nНачинаем мы приготовление тюри с того, что...",первое,02.03.2011,photo_1000menu_4.jpg,https://1000.menu/cooking/5085-turya,"[{'product_id': 0, 'name_source': 'Квас', 'uni..."
4,4,Фасолевый суп из красной фасоли,"[{'Вода': 0.3, 'unit': 'л'}, {'Картошка': 0.3,...",варка,Подготовить ингредиенты. Для приготовления суп...,первое,28.01.2013,photo_1000menu_5.jpg,https://1000.menu/cooking/38765-fasolevyi-sup-...,"[{'product_id': 828, 'name_source': 'Вода', 'u..."
...,...,...,...,...,...,...,...,...,...,...
27879,26468,Салат с филе тунца и огурцами в азиатской…,"[{'Тунец': 50.0, 'unit': 'г'}, {'Огурцы': 50.0...",жарка,1. Для приготовления азиатского соуса смешать ...,салат,20.05.2015,photo_eda_28463.jpg,https://eda.ru/recepty/salaty/salat-s-file-tun...,"[{'product_id': 5989, 'name_source': 'Тунец', ..."
27880,26469,Теплый салат с куриной печенью и яблоками,"[{'Куриная печень': 120.0, 'unit': 'г'}, {'Кис...","варка,жарка,запекание","1. Маленькими ломтиками порезать яблоки, запек...",салат,27.09.2018,photo_eda_28464.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-k...,"[{'product_id': 0, 'name_source': 'Куриная печ..."
27881,26470,Самый зеленый салат,"[{'Мангольд': 'по вкусу', 'unit': 'по вкусу'},...",сырое,"1. Чеснок натереть, сыр фета раскрошить.\r\n2....",салат,24.03.2018,photo_eda_28466.jpg,https://eda.ru/recepty/salaty/samyy-zelenyy-sa...,"[{'product_id': 3378, 'name_source': 'Мангольд..."
27882,26471,Теплый салат с тыквой и брынзой под…,"[{'Тыква': 100.0, 'unit': 'г'}, {'Сыр брынза':...",запекание,"1. Тыкву нарезаем кубиками, добавляем оливково...",салат,14.05.2019,photo_eda_28467.jpg,https://eda.ru/recepty/salaty/teplyy-salat-s-t...,"[{'product_id': 6023, 'name_source': 'Тыква', ..."


In [41]:

# Преобразование списка словарей в объект Pandas DataFrame
data = pd.DataFrame(df_rec, columns = ['name', 'Инструкции'])

# Просмотр первых 5 строк датафрейма
print(data.head(5))

                                                name  \
0  рассольник классический с перловкой и солеными...   
1                    Суп пюре из белокочаной капусты   
2                     Постные щи из квашеной капусты   
3                  Тюря- простой суп быстро и вкусно   
4                    Фасолевый суп из красной фасоли   

                                          Инструкции  
0  Подготовить указанные ингредиенты для приготов...  
1  Необходимые ингредиенты\r\nНарезаем лук, морко...  
2  Честно признаюсь, у меня не было репы на момен...  
3  \r\nНачинаем мы приготовление тюри с того, что...  
4  Подготовить ингредиенты. Для приготовления суп...  


In [42]:
train_q, test_q, train_ans, test_ans = train_test_split(data.name,
                                                        data.Инструкции,
                                                        test_size=.2,
                                                        random_state=42)

In [11]:
device = torch.device("cuda")

Модуль AutoModelForCausalLM и модуль AutoModelForQuestionAnswering являются двумя разными модулями, предназначенными для различных задач.

Модуль AutoModelForCausalLM предназначен для создания автоматически настраиваемой модели для генерации последовательностей, где последовательности имеют причинно-следственную связь. Например, такая модель может использоваться для генерации продолжения рассказа или прогнозирования следующего слова в текстовой последовательности.

С другой стороны, модуль AutoModelForQuestionAnswering предназначен для создания автоматически настраиваемой модели для ответа на вопросы на основе контекстных данных. Например, такая модель может использоваться для отвечения на вопросы в рамках задачи вопросно-ответной системы, в которой нужно найти ответ на вопрос в текстовом документе или корпусе данных.

In [12]:
model_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [43]:
def preproc_text(text):
    res = str(text).strip()
    res = re.sub(r"\s+", " ", res)
    return res

def build_data(data_q, data_ans):
    data = []
    for idx, texts in enumerate(data_q):
        question = preproc_text(texts)
        answer = preproc_text(data_ans.iloc[idx])
        res = '\nx:' + question + '\ny:' + answer
        data.append(res)
    return data

In [44]:
train_data = build_data(train_q, train_ans)
test_data = build_data(test_q, test_ans)

Этот код устанавливает токен, который будет использоваться при заполнении (padding) последовательностей текстовых данных при обучении нейронных сетей. Это делается с помощью установки значения eos_token (токена конца последовательности) как значения pad_token (токена-заполнителя).

Обычно в обучающем наборе данных бывают последовательности, которые имеют разную длину. Для обеспечения удобства и эффективности обучения модели, все последовательности должны быть одинаковой длины. Один из подходов к достижению этой цели - добавление токенов-заполнителей в конец последовательности до тех пор, пока ее длина не станет равной максимальной длине в обучающем наборе данных.

Установка pad_token в значение eos_token означает, что при заполнении последовательности токеном-заполнителем будет использоваться токен, обозначающий конец последовательности. Это может быть полезно, когда необходимо предотвратить модель от генерации ненужной информации после конца последовательности, ведь такие лишние токены могут негативно сказаться на точности модели.

In [15]:
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
#tokenizer.add_special_tokens({'pad_token': '[PAD]'})

Токенизируем тренировочный и тестовый датасеты:

In [45]:
train_encodings = tokenizer(train_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')
test_encodings = tokenizer(test_data, padding='max_length', truncation=True, max_length=512, return_tensors='pt')

In [46]:
class QAMedicalDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = QAMedicalDataset(train_encodings)
test_dataset = QAMedicalDataset(test_encodings)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [49]:
# Настройка тренировочных аргументов
training_args = TrainingArguments(
    output_dir="./models", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    )

# Задание значения max_steps перед инициализацией тренера
#training_args.max_steps = len(train_dataset) // training_args.per_device_train_batch_size * training_args.num_train_epochs

# Определение тренера
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
    eval_dataset=test_dataset
)




In [50]:
# Обучение модели
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-46-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
500,1.959400
1000,1.815300
1500,1.800800
2000,1.786300
2500,2.052700
3000,2.067900
3500,2.041500
4000,2.015200
4500,1.997600
5000,1.984500


<ipython-input-46-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-46-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-46-b29274360c54>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-46-b29274360c54>:6: UserWarning: To copy construct from a 

TrainOutput(global_step=16731, training_loss=1.8335999932350968, metrics={'train_runtime': 10014.787, 'train_samples_per_second': 6.682, 'train_steps_per_second': 1.671, 'total_flos': 1.7485924073472e+16, 'train_loss': 1.8335999932350968, 'epoch': 3.0})

Попробовала дообучить rubert-tiny2 на датасете blinoff/medical_qa_ru_data.
Обучать на CPU это боль)) в колабе не смогла дождаться, обучение прервалось через 6 часов. Сократила до 2-х эпох и урезала датасет. Взяла всего 10%, передала на cuda, обучила минут за 10-ть.  Дообученная модель выдает слишком короткие ответы.

Сомневаюсь в правильности подготовки данных (вопрос-ответ) для обучения и вывода предсказаний.

In [20]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [21]:
path =  "/content/drive/My Drive/Data Science/nlp/project/"

Сохраним модель на диск:

In [51]:
tokenizer.save_pretrained(path + 'tokenizer_qa_рецепт_gen')
model.save_pretrained(path + 'model_qa_рецепт_gen')

In [52]:
tokenizer = AutoTokenizer.from_pretrained(path + 'tokenizer_qa_рецепт_gen')
model = AutoModelForCausalLM.from_pretrained(path + 'model_qa_рецепт_gen')

In [53]:
tokenizer = AutoTokenizer.from_pretrained(path + 'tokenizer_qa_рецепт_gen')
model = AutoModelForCausalLM.from_pretrained(path + 'model_qa_рецепт_gen')

Функция для ведения диалога. Модель "держит в голове" контекст последних 10-ти сообщений.

In [54]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens,
        eos_token_id=end_token_id,
        do_sample=True,
        max_length=size+128,
        repetition_penalty=3.2,
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = re.findall(r'\ny:(.+)', decoded)[-1]
    return result.strip()

Протестируем генератор:

In [55]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'Стоп':
        break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
борщ
1. Картофель очистить, нарезать кубиками и отварить в подсоленной воде до готовности. 2. Морковь натереть на крупной терке, лук мелко порубить. 3. Свеклу почистить, потереть на крупной терке, добавить к картофелю. 4. Все перемешать, посолить, заправить майонезом. 5. Готовый борщ посыпать рубленой зеленью. 6. Подавать со сметаной. 7. Приятного аппетита! :)Приятного аппетита! :)Приятного аппетита! :)Приятного аппетита! :)Приятного аппетита! :)Приятного
оладушки
1. Яйца сварить вкрутую, отделить белки от желтков. Белки взбить с солью и перцем. Желтки растереть с сахаром. Добавить просеянную муку, смешанную с разрыхлителем. Перемешать. Влить растопленное сливочное масло. Хорошо вымесить тесто. Тесто должно получиться как густая сметана. 8. Сформировать из теста шар, завернуть в пленку и убрать в холодильник на 30 минут. 9. Раскатать тесто в пласт толщиной 0,5–0,7 мм. 10. Вырезать кружочки диаметром примерно 20 см. 11
блины
1. На сковороде разогреть 

Вроде работает ) не так хорошо как хотелось бы, конечно